In [147]:
%load_ext autoreload
%autoreload 2
    
import numpy as np
from subprocess import getoutput
import requests
import sys
import ssl
import pandas as pd
from OpenSSL import crypto
import matplotlib.pyplot as plt
import pandas as pd
import os
from Crypto.PublicKey import RSA
from batch_gdc import *
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [148]:
pd_key = pd.read_csv('../03/domeny_klucze.csv')
#pd_key = pd_key.dropna()

### Key to number

In [4]:
k = '-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAubanqNe6xSQfWiszYK6E\nq/7AgBk8lWhyol87YWFU/sQU56lRRcBuUE6wzhNw8ZlyqbaTB+Lui0/JQ9qE0yIM\ngO9ll8Vy+5YgCDEHGevMQJF/TUAUM1yv1SWlTz0DO2+Yu0t8sjL9wDleqUbJ99PF\nChDxosgx+2STU4LC6o238znnTmw2OKz7UU1z1BpImaYw7jJLt89kU0LyNf2yWKNh\n7iHPD7adPjpaz0Cnc4JzLzESwlWYMPwVevFAwyYaT7CEm+RAjyQH/2gTC/cjD8od\nEliNHXDbRVMnFtgDvDccp4v5wKxG4fo+7WqFkEKjYrreUDw8M9SiRf00VTaqpXEH\n3wIDAQAB\n-----END PUBLIC KEY-----'
#k[30:-29]
k_int = int.from_bytes(k[27:-25].encode(),'little')

In [5]:
pub_key = RSA.importKey(k)

In [6]:
print('N',pub_key.n)
print('E',pub_key.e)

N 23444156916788217072968948797613723278925042381586582221140693672863149672676005620893979925204494030744622095025905422202071522522127578033612221016275813745156551118151270658253026197529489724236950117170556081137622165601495755640204329543647366736864367886398715741997966193998377964536823291966611064944123521230452486637382460409243307967480693978253936290404185984017525643259622532424222237847334527637929682299248260640852007183565826447648165000289210266187323013336594237477719650639394912377218794510735697325898093989060223722951869207292817471564468215796618201620610540272670321646987987450753140721631
E 65537


# BatchGCD

In [149]:
pd_key = pd_key.loc[pd_key['Key'].notnull()]
pd_key.head()
#pd_key.drop()

,Unnamed: 0,Domain,OpenRank,Key
0,0,mattpresti.com,5.56,b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG...
1,1,mattsteady.com,5.56,b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG...
2,2,maximalsound.com,5.56,b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG...
3,3,mayapuri.com,5.56,b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG...
4,4,mcphillipstoyota.com,5.56,b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG...


In [150]:
N = []
for ii, p in tqdm(pd_key.iterrows(), total=pd_key.shape[0]):
    try:
        key = p['Key'][2:-1].replace('\\n','\n')
        RSA_key = RSA.importKey(key)
        N.append(RSA_key.n)
    except:
        N.append(0)

100%|██████████| 167421/167421 [00:44<00:00, 3767.62it/s]


In [153]:
N = np.array(N)
zera = np.where(N==0)[0]
pd_key.drop(zera)

,Unnamed: 0,Domain,OpenRank,Key
1,1,mattsteady.com,5.56,b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG...
2,2,maximalsound.com,5.56,b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG...
3,3,mayapuri.com,5.56,b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG...
4,4,mcphillipstoyota.com,5.56,b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG...
5,5,store.meater.com,5.56,b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG...
...,...,...,...,...
233992,233992,mysocialpoker.com,5.56,b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG...
233993,233993,naomifindlay.com,5.56,b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG...
233995,233995,natinstablog.com,5.56,b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG...
233997,233997,natpemarket.com,5.56,b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG...


In [164]:
#N = np.trim_zeros(N)
N = np.unique(N)
N = N[np.where(N!=0)[0]]
print(len(N))

72052


In [117]:
#pd_key['N'] = np.array(N).astype(np.str) #np.zeros(len(N)).astype(np.int32)

ValueError: Length of values does not match length of index

In [ ]:
bath = batchgcd_faster(N[-3000:])

In [173]:
print(bath)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 